<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/train.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/test.txt'
!wget -q --show-progress 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/dev.txt'
!mkdir data && mv train.txt data && mv test.txt data && mv dev.txt data

train.txt           100%[===================>]   2.20M  --.-KB/s    in 0.05s   
test.txt            100%[===================>] 108.10K  --.-KB/s    in 0.01s   
dev.txt             100%[===================>] 160.83K  --.-KB/s    in 0.01s   


# Benchmark2

In [3]:
!cat data/train.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > train.txt.tmp
!cat data/test.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > test.txt.tmp
!cat data/dev.txt | grep -v "^#" | cut -f 1,2 | tr '\t' ' ' > dev.txt.tmp

In [4]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 62794 (delta 5), reused 14 (delta 4), pack-reused 62772
Receiving objects: 100% (62794/62794), 47.88 MiB | 23.18 MiB/s, done.
Resolving deltas: 100% (44467/44467), done.


In [5]:
!pip -q install transformers/
!pip -q install sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 7.6MB/s 
     |████████████████████████████████| 3.2MB 21.6MB/s 
     |████████████████████████████████| 1.2MB 10.1MB/s 


In [6]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2021-02-19 06:10:34--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2021-02-19 06:10:34 (53.3 MB/s) - ‘preprocess.py’ saved [991/991]



In [7]:
!python3 preprocess.py dev.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > dev.txt
!python3 preprocess.py train.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > train.txt
!python3 preprocess.py test.txt.tmp HooshvareLab/bert-base-parsbert-uncased 256 > test.txt

Downloading: 100% 434/434 [00:00<00:00, 499kB/s]
Downloading: 100% 1.22M/1.22M [00:00<00:00, 3.14MB/s]


In [8]:
!pip -q install -r transformers/examples/token-classification/requirements.txt

     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 184kB 11.5MB/s 
     |████████████████████████████████| 20.7MB 1.4MB/s 
     |████████████████████████████████| 245kB 21.8MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 


In [9]:
!pip -q install conllu

In [10]:
!mkdir processed_data
!mv train.txt processed_data/ && mv test.txt processed_data/ && mv dev.txt processed_data/

In [11]:
!cat processed_data/train.txt processed_data/test.txt processed_data/dev.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt

In [15]:
! cp /content/transformers/examples/legacy/token-classification/utils_ner.py /content/transformers/examples/token-classification
! cp /content/transformers/examples/legacy/token-classification/tasks.py /content/transformers/examples/token-classification

In [17]:
!python3 transformers/examples/token-classification/run_tf_ner.py \
--data_dir ./processed_data/ \
--labels labels.txt \
--model_name_or_path HooshvareLab/bert-base-parsbert-uncased \
--output_dir eval/ \
--max_seq_length  256 \
--per_device_train_batch_size 8 \
--save_steps 750 \
--seed 1 \
--do_train \
--do_eval \
--do_predict

2021-02-19 06:18:28.055314: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
[INFO|training_args.py:566] 2021-02-19 06:18:29,296 >> PyTorch: setting up devices
[INFO|training_args.py:504] 2021-02-19 06:18:29,335 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|training_args_tf.py:178] 2021-02-19 06:18:29,341 >> Tensorflow: setting up strategy
2021-02-19 06:18:29.342056: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-19 06:18:29.342163: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-19 06:18:29.342303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.c

In [18]:
!zip -r eval.zip eval

  adding: eval/ (stored 0%)
  adding: eval/test_results.txt (deflated 65%)
  adding: eval/tokenizer_config.json (deflated 45%)
  adding: eval/config.json (deflated 54%)
  adding: eval/tf_model.h5 (deflated 8%)
  adding: eval/eval_results.txt (deflated 23%)
  adding: eval/checkpoint/ (stored 0%)
  adding: eval/checkpoint/ckpt-1.data-00000-of-00001 (deflated 31%)
  adding: eval/checkpoint/checkpoint (deflated 63%)
  adding: eval/checkpoint/ckpt-1.index (deflated 82%)
  adding: eval/checkpoint/ckpt-3.index (deflated 82%)
  adding: eval/checkpoint/ckpt-2.index (deflated 82%)
  adding: eval/checkpoint/ckpt-3.data-00000-of-00001 (deflated 30%)
  adding: eval/checkpoint/ckpt-2.data-00000-of-00001 (deflated 30%)
  adding: eval/special_tokens_map.json (deflated 40%)
  adding: eval/test_predictions.txt (deflated 71%)
  adding: eval/vocab.txt (deflated 62%)


In [19]:
!zip -r runs.zip runs

  adding: runs/ (stored 0%)
  adding: runs/Feb19_06-14-29_152726cab548/ (stored 0%)
  adding: runs/Feb19_06-14-29_152726cab548/events.out.tfevents.1613715314.152726cab548.282.4401.v2 (deflated 55%)
  adding: runs/Feb19_06-18-29_152726cab548/ (stored 0%)
  adding: runs/Feb19_06-18-29_152726cab548/events.out.tfevents.1613715529.152726cab548.354.4401.v2 (deflated 56%)


In [20]:
!ls -lh

total 4.4G
drwxr-xr-x  2 root root 4.0K Feb 19 06:10 data
-rw-r--r--  1 root root 161K Feb 19 06:10 dev.txt.tmp
drwxr-xr-x  3 root root 4.0K Feb 19 06:38 eval
-rw-r--r--  1 root root 4.4G Feb 19 06:45 eval.zip
-rw-r--r--  1 root root   74 Feb 19 06:11 labels.txt
-rw-r--r--  1 root root  991 Feb 19 06:10 preprocess.py
drwxr-xr-x  2 root root 4.0K Feb 19 06:11 processed_data
drwxr-xr-x  4 root root 4.0K Feb 19 06:18 runs
-rw-r--r--  1 root root 3.3K Feb 19 06:45 runs.zip
drwxr-xr-x  1 root root 4.0K Feb 16 16:35 sample_data
-rw-r--r--  1 root root 109K Feb 19 06:10 test.txt.tmp
-rw-r--r--  1 root root 2.3M Feb 19 06:10 train.txt.tmp
drwxr-xr-x 15 root root 4.0K Feb 19 06:10 transformers


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
!mv eval.zip drive/MyDrive/

In [23]:
!mv runs.zip drive/MyDrive/